In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
print(tf.__version__)

!mkdir /root/.kaggle/
# with open("/root/.kaggle/kaggle.json", "w") as file:
#     file.write('{"username":"burindima","key":"---"}')
!kaggle competitions download -c ml-intensive-of-yandex-academy-2023
!unzip ml-intensive-of-yandex-academy-2023.zip 

train_data = pd.read_pickle('data_train')
test_data = pd.read_pickle('data_test')

print(f'Train data keys: {train_data.keys()}')
print(f'Test data keys: {test_data.keys()}\n')
print(f'Shape of train images: {train_data["images"].shape}')
print(f'Shape of test images: {test_data["images"].shape}\n')
print(f'Unique labels: {set(train_data["labels"])}\n')
print(f'Classes distribution: {pd.Series(train_data["labels"]).value_counts(normalize=True)}')

2.12.0
 97% 255M/263M [00:02<00:00, 141MB/s] 
100% 263M/263M [00:02<00:00, 99.1MB/s]
Archive:  ml-intensive-of-yandex-academy-2023.zip
  inflating: data_test               
  inflating: data_train              
  inflating: meta                    
Train data keys: dict_keys(['section', 'labels', 'images'])
Test data keys: dict_keys(['section', 'images'])

Shape of train images: (50000, 32, 32, 3)
Shape of test images: (25000, 32, 32, 3)

Unique labels: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

Classes distribution: 1    0.19300
2    0.15190
3    0.11250
4    0.09882
5    0.09208
7    0.07752
6    0.07718
0    0.06682
8    0.06664
9    0.06354
dtype: float64


In [ ]:
print(50000* 0.15190)
print(50000*0.19300)

7595.0
9650.0


## ЧБ

In [ ]:
def black_white_point(image):
    # получаем координаты черных пикселей
  black_pixels = np.where(np.all(image == [0, 0, 0], axis=-1))
  # print(black_pixels)

  # заменяем черные пиксели на среднее значение пикселей в окрестности 3x3
  for i, j in zip(black_pixels[0], black_pixels[1]):
      # получаем окрестность 3x3 пикселя
      neighborhood = image[max(0, i-1):min(i+2, image.shape[0]), max(0, j-1):min(j+2, image.shape[1])]
      # отсеиваем соль и перец чтобы не мешались
      neighborhood1 = neighborhood.reshape((-1, 3))
      neighborhood1 = neighborhood1[np.nonzero(np.sum(neighborhood1, axis=1))]
      neighborhood1 = neighborhood1[np.nonzero(np.sum(neighborhood1, axis=1) != 255*3)]
      if not neighborhood1.any():  # на случай если пиксель полностью окружён шумом
          neighborhood1 = neighborhood
      # считаем среднее значение пикселей
      mean_pixel = np.mean(neighborhood, axis=(0, 1))
      # заменяем черный пиксель на среднее значение
      image[i, j] = mean_pixel


  # получаем координаты белых пикселей
  white_pixels = np.where(np.all(image == [255, 255, 255], axis=-1))
  # print(white_pixels)

  # заменяем черные пиксели на среднее значение пикселей в окрестности 3x3
  for i, j in zip(white_pixels[0], white_pixels[1]):
      # получаем окрестность 3x3 пикселя
      neighborhood = image[max(0, i-1):min(i+2, image.shape[0]), max(0, j-1):min(j+2, image.shape[1])]
      # отсеиваем соль и перец чтобы не мешались
      neighborhood1 = neighborhood.reshape((-1, 3))
      neighborhood1 = neighborhood1[np.nonzero(np.sum(neighborhood1, axis=1))]
      neighborhood1 = neighborhood1[np.nonzero(np.sum(neighborhood1, axis=1) != 255*3)]
      if not neighborhood1.any():  # на случай если пиксель полностью окружён шумом
          neighborhood1 = neighborhood
      # считаем среднее значение пикселей
      mean_pixel = np.mean(neighborhood, axis=(0, 1))
      # заменяем белый пиксель на среднее значение
      image[i, j] = mean_pixel

In [ ]:
for image in train_data['images']:
  black_white_point(image)

for image in test_data['images']:
  black_white_point(image)

## АУГМ

In [ ]:
train_data

In [ ]:
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()


data_augmentation = tf.keras.Sequential([
  # layers.RandomFlip("horizontal"),
  layers.RandomRotation(0.1),
  layers.RandomContrast(0.4),
  layers.RandomBrightness(0.2),
  layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
])



In [ ]:
orig_ds = tf.data.Dataset.from_tensor_slices((train_data['images'], train_data['labels']))
print(len(orig_ds))

50000


In [ ]:
augment_ds = orig_ds
# ко всем цифрам 0, 8, 9 будут прибавлены по две аугментированной
dict_augment_for_label = {
    0: 2,
    8: 2,
    9: 2
}
for label, value in dict_augment_for_label.items():
  for i in range(value):
    certain_label = orig_ds.filter(lambda x, y: y == label)
    aug_ds = certain_label.map(
    lambda x, y: (data_augmentation(x, training=True), y))
    augment_ds = augment_ds.concatenate(aug_ds)

In [ ]:
dict_augment_for_label = {
    7: 2,
    6: 2,
}
# к 75% цифр 7,6 будут прибавлены аугментированные
percent_augment_label = 0.75
for label, value in dict_augment_for_label.items():
  for i in range(value):
    certain_label = orig_ds.filter(lambda x, y: y == label).shuffle(1000)
    certain_label = certain_label.take(int(len([i for i in certain_label]) * percent_augment_label))
    aug_ds = certain_label.map(
    lambda x, y: (data_augmentation(x, training=True), y))
    augment_ds = augment_ds.concatenate(aug_ds)


In [ ]:
# ко всем цифрам 4, 5 будут прибавлены по одной аугментированной
dict_augment_for_label = {
    4: 1,
    5: 1,
}
for label, value in dict_augment_for_label.items():
  for i in range(value):
    certain_label = orig_ds.filter(lambda x, y: y == label)
    aug_ds = certain_label.map(
    lambda x, y: (data_augmentation(x, training=True), y))
    augment_ds = augment_ds.concatenate(aug_ds)

In [ ]:
dict_augment_for_label = {
    3: 1,
}
# к 70% цифр 3 будут прибавлены аугментированные
percent_augment_label = 0.7
for label, value in dict_augment_for_label.items():
  for i in range(value):
    certain_label = orig_ds.filter(lambda x, y: y == label).shuffle(1000)
    certain_label = certain_label.take(int(len([i for i in certain_label]) * percent_augment_label))
    aug_ds = certain_label.map(
    lambda x, y: (data_augmentation(x, training=True), y))
    augment_ds = augment_ds.concatenate(aug_ds)


In [ ]:
dict_augment_for_label = {
    2: 1,
}
# к 27% цифр 2 будут прибавлены аугментированные
percent_augment_label = 0.27
for label, value in dict_augment_for_label.items():
  for i in range(value):
    certain_label = orig_ds.filter(lambda x, y: y == label).shuffle(1000)
    certain_label = certain_label.take(int(len([i for i in certain_label]) * percent_augment_label))
    aug_ds = certain_label.map(
    lambda x, y: (data_augmentation(x, training=True), y))
    augment_ds = augment_ds.concatenate(aug_ds)


In [ ]:
len_orig_ds = len([i for i in augment_ds])

In [ ]:
augment_ds

In [ ]:
orig_ds = augment_ds
orig_val_ds, orig_train_ds = orig_ds.take(int(len_orig_ds* 0.05)) , orig_ds.skip(int(len_orig_ds * 0.05))
# print(len(orig_val_ds), len(orig_train_ds))
print(len([i for i in orig_val_ds]))

In [ ]:
batch_size = 1024
AUTOTUNE = tf.data.AUTOTUNE
IMG_SIZE = 32

resize_and_rescale = tf.keras.Sequential([
  layers.Rescaling(1./255)
])

def prepare(ds, shuffle=False, augment=False):
  # Resize and rescale all datasets.
  ds = ds.map(lambda x, y: (resize_and_rescale(x), y), 
              num_parallel_calls=AUTOTUNE)

  if shuffle:
    ds = ds.shuffle(1000)

  # Batch all datasets.
  ds = ds.batch(batch_size).map(lambda x, y: (x, tf.one_hot(y, depth=10)))

  # Use data augmentation only on the training set.
  if augment:
    ds = ds.map(lambda x, y: (data_augmentation(x, training=True), y), 
                num_parallel_calls=AUTOTUNE)

  # Use buffered prefetching on all datasets.
  return ds.prefetch(buffer_size=AUTOTUNE)

train_ds = prepare(orig_train_ds, shuffle=True)
val_ds = prepare(orig_val_ds)
# test_ds = prepare(test_ds)

In [ ]:
from tensorflow.keras import layers

#СОЗДАЕМ МОДЕЛЬ 
input_shape = train_data['images'].shape[-3:]

import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D, BatchNormalization, Activation, Add,Dropout


# Определение функции для создания Residual блоков
def residual_block(inputs, num_filters, kernel_size, strides):
    x = Conv2D(num_filters, kernel_size=kernel_size, strides=strides, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Dropout(0.4)(x)
    x = Activation('relu')(x)
    x = Conv2D(num_filters, kernel_size=kernel_size, strides=strides, padding='same')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.4)(x)
    x = Add()([x, inputs])
    x = Activation('relu')(x)
    return x

# Создание модели ResNet-50 архитектуры
inputs = Input(shape=input_shape)
x = Conv2D(64, kernel_size=3, strides=1, padding='same')(inputs)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = residual_block(x, 64, 3, 1)
x = residual_block(x, 64, 3, 1)
x = residual_block(x, 64, 3, 1)
x = Conv2D(128, kernel_size=3, strides=2, padding='same')(x)
x = BatchNormalization()(x)
x = Dropout(0.4)(x)
x = Activation('relu')(x)
x = residual_block(x, 128, 3, 1)
x = residual_block(x, 128, 3, 1)
x = residual_block(x, 128, 3, 1)
x = Conv2D(256, kernel_size=3, strides=2, padding='same')(x)
x = BatchNormalization()(x)
x = Dropout(0.4)(x)
x = Activation('relu')(x)
x = residual_block(x, 256, 3, 1)
x = residual_block(x, 256, 3, 1)
x = residual_block(x, 256, 3, 1)
x = Flatten()(x)
outputs = Dense(10, activation='softmax')(x)
model = Model(inputs, outputs)


# Компиляция и обучение модели
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
!pip install livelossplot

In [ ]:
from livelossplot import PlotLossesKeras
import time 

start = time.time() 
model_history = model.fit(train_ds, validation_data=val_ds, epochs=100, callbacks=[PlotLossesKeras()], verbose=False)
end = time.time() - start 
print("Время: "+ str(end) +' секунд')